<a href="https://colab.research.google.com/github/sridhartroy/AIML/blob/main/CS6350_Assignment2_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkContext
import random

# Initialize SparkContext
sc = SparkContext.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 19:54:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#Download the data from the url above and create an RDD called friends
!wget https://an-ml.s3.us-west-1.amazonaws.com/soc-LiveJournal1Adj.txt -O friends_part1.txt

--2025-10-26 19:54:52--  https://an-ml.s3.us-west-1.amazonaws.com/soc-LiveJournal1Adj.txt
Resolving an-ml.s3.us-west-1.amazonaws.com (an-ml.s3.us-west-1.amazonaws.com)... 52.219.194.114, 16.15.4.75, 16.15.0.18, ...
Connecting to an-ml.s3.us-west-1.amazonaws.com (an-ml.s3.us-west-1.amazonaws.com)|52.219.194.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156181 (4.0M) [text/plain]
Saving to: ‘friends_part1.txt’

friends_part1.txt   100%[===================>]   3.96M  7.71MB/s    in 0.5s    

2025-10-26 19:54:52 (7.71 MB/s) - ‘friends_part1.txt’ saved [4156181/4156181]



In [ ]:
# Read the friends data using Spark Context
friends_file = sc.textFile("friends_part1.txt")

# Parse the data into (user, set_of_friends) format
# Remove any users that have no friends listed
user_friends = friends_file.map(lambda line: line.split('\t')) \
                    .filter(lambda x: len(x) == 2 and x[1].strip() != '') \
                    .map(lambda x: (int(x[0]), set(map(int, x[1].split(',')))))

# Print sample data
print("Sample user-friends data:")
for user, friends in user_friends.take(10):
    print(f"User {user}: Friends {friends}")

Sample user-friends data:


User 0: Friends {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94}
User 1: Friends {0, 31232, 29826, 35589, 5, 135, 4999, 34439, 8715, 6156, 13840, 19217, 27536, 34450, 20, 13845, 35605, 35606, 22939, 29724, 35613, 29726, 28193, 33060, 14248, 29481, 12846, 44846, 35633, 30257, 21556, 14005, 43447, 49592, 12347, 32317, 35648, 49985, 29767, 49226, 31435, 34250, 38737, 15190, 44887, 13912, 34392, 34394, 34651, 35678, 29791, 23520, 30691, 8932, 17636, 34406, 34280, 2409, 32489, 20075, 20074, 623, 34418, 34420, 629, 45054, 10623}
User 2: Friends {0, 2755, 1220, 12453, 13795, 135, 49927, 24714, 6893, 41456, 16659, 117, 45046, 41878, 24539, 32828}
User 3: Friends {0, 13185, 27552, 

In [ ]:
# Create a function to generate mutual friend pairs, given a user and their list of friends
# The key is the pair of friends, and the value is 1 (indicating one mutual friend)
def generate_mutual_friend_pairs(user, friends):
    pairs = []
    for f1 in friends:
        for f2 in friends:
            if f1 != f2:
                pairs.append(((f1, f2), 1))
    return pairs

In [ ]:
# Use the generated function to create an RDD of mutual friend pairs with an appended 1
# Then use a flatmap to flatten the list of pairs
mutual_friends = user_friends.flatMap(lambda x: generate_mutual_friend_pairs(x[0], x[1]))

# Print a subset of mutual friend pairs
print("Subset of mutual friend pairs:")
for pair, count in mutual_friends.take(10):
    print(f"[{pair}: {count}]")

Subset of mutual friend pairs:
[(1, 2): 1]
[(1, 3): 1]
[(1, 4): 1]
[(1, 5): 1]
[(1, 6): 1]
[(1, 7): 1]
[(1, 8): 1]
[(1, 9): 1]
[(1, 10): 1]
[(1, 11): 1]


In [ ]:
# Use the reduceByKey transformation to count mutual friends for each pair
# This count will serve as the recommendation score
mutual_scores = mutual_friends.reduceByKey(lambda x, y: x + y)

# Print subset of mutual friend counts
#print("Subset of mutuals and scores")
#for pair, count in mutual_scores.take(10):
    #print(f"[{pair}: {count}]")

In [ ]:
# Create an RDD of existing friendships to filter out
existing_friendships = user_friends.flatMap(lambda x: [(x[0], friend) for friend in x[1]])

# Print subset existing friendships
#print("Subset of existing friendships:")
#for user, friend in existing_friendships.take(10):
#    print(f"({user}, {friend})")

In [ ]:
# Make the key as a tuple of the friend pair and value as None
# This will help in filtering out existing friends later
existing_friend_set = existing_friendships.map(lambda x: ((x[0], x[1]), None))

# Print new RDD with pairs as key
#print("Subset existing friend pairs:")
#for pair, _ in existing_friend_set.take(10):
#    print(f"[{pair}, {_}]")

In [ ]:
# Filter out existing friends from potential recommendations
# Use subtractByKey to remove existing friendships from mutual scores
# This will leave only potential new friend recommendations
# Output is ((user1, user2), mutual_score)
potential_recommendations = mutual_scores.subtractByKey(existing_friend_set)

# Print subset potential recommendations
#print("Subset of potential recommendations:")
#for pair, count in potential_recommendations.take(20):
#    print(f"({pair} {count})")

In [ ]:
# Initila RDD is ((user1, user2), mutual_score)
# Format the recommendations as (user, (recommended_friend, mutual_score))
recommendations = potential_recommendations.map(lambda x: (x[0][0], (x[0][1], x[1])))

# Print Subset friend recommendations for users
#print("Subset friend recommendations:")
#for user, rec in recommendations.take(20):
#    print(f"User {user}: Recommendation {rec}")

In [ ]:
# Get top 10 recommendations for each user
# Group the recommendations by user
# Using mapValues, sort the recommendations in decsending order of mutual score and take the top ten
# From the resulting list, use mapValues to extract only the recommended user IDs as strings in an array
# Output is (user, [recommended_user1, recommended_user2, ..., recommended_user10])
top_recommendations = recommendations.groupByKey() \
    .mapValues(lambda recs: sorted(recs, key=lambda x: -x[1])[:10]) \
    .mapValues(lambda recs: [str(user_id) for user_id, count in recs]).cache()

In [ ]:
# Get a sample of 10 users and their top recommendations
sample_users = top_recommendations.takeSample(False, 10)

print("Sample top recommendations for 10 users:")
for user, recs in sample_users:
    print(f"{user}\t{','.join(recs)}")

Sample top recommendations for 10 users:
33155	11743,11714,33306,6523,7352,24498,11746,37883,33135,11723
47804	42897,17489,31024,11863,15757,45992,18848,43016,36659,20846
47117	47054,47124,47125,47157,47086,47047,47087,47043,47128,47044
24491	20869,24482,24475,24487,6935,24495,24483,24479,24480,24488
17326	40172,17324,17292,17300,17296,17308,17312,17304,17316,12160
15592	15550,15538,15498,15542,15566,15546,15562,15558,15574,15582
35259	35199,35212,35243,35183,35239,35255,35184,35228,35236,35260
17045	17073,12313,17058,9362,15130,17059,551,23380,11244,5225
43075	12351,8671,40455,4439,8695,37955,24595,8635,7847,40443
2800	2798,2814,2826,2806,2830,2822,2786,2790,2794,2802
